In [ ]:
#| default_exp routers/deck

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from fastapi import APIRouter

In [ ]:
#| export
router = APIRouter()

In [ ]:
#| export
from typing import List, Literal
from pydantic import BaseModel 
import random

RANKS = '5 9 J Q K A'.split()
SUITS = '♣ ♢ ♡ ♠'.split()
values = {'5': 10, '9': 0, 'J': 2, 'Q': 3, 'K': 4, 'A': 11}

class PlayingCard(BaseModel):
    rank: Literal['9', 'J', 'Q', 'K', '5', 'A']
    suit: Literal['♣', '♢', '♡', '♠']
    value: int

    @property
    def value(self):
        return values[self.rank]

    def __repr__(self):
        return f'{self.suit}{self.rank}'

    def __lt__(self, other):
        return self.value < other.value

class Deck:
    def __init__(self, cards: List[PlayingCard]):
        self.cards = cards
        random.shuffle(self.cards)
        self.is_shuffled = True
        self._played_cards = []
        self._starting_cards = self.cards.copy()

    def __repr__(self):
        return ' '.join(str(c) for c in self.cards)

    def draw_cards(self, n: int):
        return [self.draw_card() for _ in range(n)]

    def draw_card(self):
        if len(self.cards) == 0:
            return None
        else:
            played_card = self.cards.pop()
            self._played_cards.append(played_card)
            return played_card

class Hand(BaseModel):
    cards: List[PlayingCard]

    def add_card(self, card: PlayingCard):
        self.cards.append(card)

    def remove_card(self, card: PlayingCard):
        self.cards.remove(card)

    def sort_cards(self):
        self.cards.sort()

    def __str__(self):
        return ' '.join(str(c) for c in self.cards)


def make_idiot_deck():
    return [PlayingCard(rank=r, suit=s) for s in SUITS for r in RANKS]

deck = Deck(cards=sorted(make_idiot_deck()))

In [ ]:
#| export
@router.get('/hand')
async def gethand() -> Hand:
    hand = Hand(cards=deck.draw_cards(5))
    return hand

In [ ]:
#| export
@router.get('/deck')
async def aroute():
    return {"message": "Welcome in aroute within deck!"}

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()